## Análise e Estimação de um Modelo para série de vendas no varejo

In [ ]:
#Bibliotecas

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import statsmodels.tsa.api as smt 

from sklearn.linear_model import LinearRegression

from numpy import polyfit

from matplotlib.pylab import rcParams


In [ ]:
#tamanho das imagens
rcParams['figure.figsize'] = 15, 6

#### Leitura de Dados

In [ ]:
dados = pd.read_excel('vendas_varejo.xlsx')

In [ ]:
dados.head(3)

In [ ]:
serie_vendas = pd.Series(dados['Indice'].values, index= dados['Data'])

In [ ]:
plt.plot(serie_vendas)
plt.show()

In [ ]:
## Verificando o comportamento de 2 ano da série
plt.plot(serie_vendas[0:24])
plt.show()

In [ ]:
smt.graphics.plot_pacf(serie_vendas, lags=12, alpha=0.5)

#### Verificando se existe tendência 

In [ ]:
m_m = serie_vendas.rolling(window=12).mean()
m_std = serie_vendas.rolling(window=12).std() 

orig = plt.plot(serie_vendas, color='blue',label='Original')
mean = plt.plot(m_m, color='red',label='Média')
std = plt.plot(m_std, color='black',label='Desvio')
plt.legend(loc='best')
plt.show()

### 1ª Resolução

* Utilizar as seguintes técnicas: 
  * Estimar tendência através de uma reta utilizando regressão linear
  * Eliminar a tendência estimada
  * Estimar a sazonalidade através de polinômios 
  * Eliminar a sazonalidade
  * Utilizar um modelo de média zero para representar o resíduo 


In [ ]:
X = [i for i in range(0, len(serie_vendas))]
X = np.reshape(X, (len(X), 1))
y = serie_vendas
modelo = LinearRegression()
modelo.fit(X, y)  

In [ ]:
modelo.coef_

In [ ]:
tendencia = modelo.predict(X)

In [ ]:
plt.plot(serie_vendas.values, color='blue',label='Original')
plt.plot(tendencia,color='red', label='Tendência' )
plt.show()

#### Eliminação de Tendência

In [ ]:
serie_sem_tendencia = serie_vendas - tendencia

In [ ]:
m_m = serie_sem_tendencia.rolling(window=12).mean()
mean = plt.plot(m_m, color='red',label='Média')
plt.plot(serie_sem_tendencia, label='Sem tendencia')
plt.legend(loc='best')
plt.show()

#### Estimando o componente de sazonalidade

In [ ]:
# ajuste polynomial: x^2*b1 + x*b2 + ... + bn
X = [i for i in range(0, len(serie_sem_tendencia))]
y = serie_sem_tendencia.values
degree = 12
coef = polyfit(X, y, degree)
print('Coefficients: %s' % coef)

In [ ]:
curva = []
for i in range(len(X)):
    valor = coef[-1]
    for d in range(degree):
        valor += X[i]**(degree-d) * coef[d]
    curva.append(valor)

In [ ]:
plt.plot(serie_sem_tendencia.values)
plt.plot(curva, color='red', label='sazonalidade')
plt.legend(loc='best')
plt.show()

#### Eliminação da sazonalide

In [ ]:
serie_sem_tendencia_sazonalidade = serie_sem_tendencia - curva

In [ ]:
m_m = serie_sem_tendencia_sazonalidade.rolling(window=12).mean()
plt.plot(serie_sem_tendencia_sazonalidade, label='resíduo')
mean = plt.plot(m_m, color='red',label='Média')
plt.legend(loc='best')
plt.show()

In [ ]:
smt.graphics.plot_pacf(serie_sem_tendencia_sazonalidade, lags=12, alpha=0.5)

#### Aplicando um modelo de média zero no resíduo 

In [ ]:
desvio = 2
random_normal = [i for i in np.random.normal(scale=desvio, size = len(serie_sem_tendencia_sazonalidade))]

In [ ]:
plt.plot(serie_sem_tendencia_sazonalidade.values, label='resíduo')
plt.plot(random_normal, color='red',label='Random estimado')
plt.legend(loc='best')
plt.show()

#### Previsão


In [ ]:
def previsao_tendencia(indices_anteriores, comp_tend):
    #prever a tendência utilizando uma reta estimada por regressão
    indices_anteriores = np.reshape(indices_anteriores, (len(indices_anteriores),1))
    
    prev_tendencia = comp_tend.predict(indices_anteriores)
    return prev_tendencia

In [ ]:
def previsao_sazonalidade(indices_anteriores, comp_tend):
    #prever o valor do componente de sazonalidade utilizando ajuste polinomial 
    curva = []
    for i in range(len(indices_anteriores)):
        valor = comp_tend[-1]
        for d in range(degree):
            valor += indices_anteriores[i]**(degree-d) * comp_tend[d]
        curva.append(valor)
    return curva
    

In [ ]:
def previsao_final(indices, comp_tend, comp_sazon, desv_random):
    #prever o valor da tendência
    tendencia = previsao_tendencia(indices, comp_tend)
    
    #prever o valor da sazonalidade
    sazonalidade = previsao_sazonalidade(indices, coef)
        
    #prever o valor do resíduo
    residuo = [i for i in np.random.normal(scale=desv_random, size = len(indices))]
    
    prev_final = tendencia + sazonalidade + residuo
    return prev_final 
    

In [ ]:
# Prevendo os dados de treinamento
indices = [i for i in range(0, len(serie_vendas))]
prev_vendas= previsao_final(indices, modelo, coef, desvio)

In [ ]:
plt.plot(serie_vendas.values, color='blue', label='Serie Real')
plt.plot(prev_vendas, color='red', label='Previsao')
plt.legend(loc='best')
plt.show()

### 2ª Resolução

* Utilizar as seguintes técnicas: 
  * Estimar tendência através de média móveis e ajuste exponencial 
  * Eliminar a tendência estimada
  * Estimar a sazonalidade através através de média móveis 
  * Eliminar a sazonalidade
  * Utilizar um modelo de média zero para representar o resíduo 
  

#### Estimando a tendência por média móveis 

In [ ]:
rolling = serie_vendas.rolling(window=12)
media_mov = rolling.mean()
# plot original and transformed dataset
plt.plot(serie_vendas, label='Série Real')
plt.plot(media_mov,color='red', label='Média Móveis')
plt.legend(loc='best')
plt.show()

#### Estimando a tendência por suavização exponencial 

In [ ]:
rolling =  serie_vendas.ewm(alpha =0.1)
suav_exp = rolling.mean()
plt.plot(serie_vendas, label='Série Real')
plt.plot(suav_exp,color='red', label='Exponencial')
plt.legend(loc='best')
plt.show()

#### Eliminando o componente de tendência

In [ ]:
serie_sem_tendencia = serie_vendas - suav_exp  # Série - componente de tendência

#### Estimando o componente de sazonalidade por média móvel 

In [ ]:
#Aplica média móveis na série sem a tendência 
rolling = serie_sem_tendencia.rolling(window=2)
saz_mean_mov = rolling.mean()
# plot original and transformed dataset
plt.plot(serie_sem_tendencia, label='Série sem tendência')
plt.plot(saz_mean_mov,color='red', label='Média Móveis')
plt.legend(loc='best')
plt.show()

#### Eliminando a sazonalidade


In [ ]:
serie_residuo = serie_sem_tendencia - saz_mean_mov #Elimina o componente de sazonalidade 

In [ ]:
m_m = serie_residuo.rolling(window=12).mean()
plt.plot(serie_residuo, label='resíduo')
mean = plt.plot(m_m, color='red',label='Média')
plt.legend(loc='best')
plt.show()

#### Aplicando um modelo de média zero no resíduo 

In [ ]:
desvio = 2
random_normal = [i for i in np.random.normal(scale=desvio, size = len(serie_residuo))]

In [ ]:
plt.plot(serie_sem_tendencia_sazonalidade.values, label='resíduo')
plt.plot(random_normal, color='red',label='Random estimado')
plt.legend(loc='best')
plt.show()

### 3ª Resolução

* Utilizar as seguintes técnicas: 
  * Eliminar a tendência através de diferenciação
  * Eliminar a sazonalidade através de diferenciação
  * Utilizar um modelo de média zero para representar o resíduo 

#### Eliminando a tendência através da Diferenciação

In [ ]:
serie_diff = []
for i in range(1, len(serie_vendas)):
    valor = serie_vendas[i] - serie_vendas[i - 1]
    serie_diff.append(valor)

serie_diff = pd.Series(serie_diff)

In [ ]:
m_m = serie_diff.rolling(window=12).mean()
plt.plot(serie_diff, label='Série com Diff')
mean = plt.plot(m_m, color='red',label='Média')
plt.legend(loc='best')
plt.show()

In [ ]:
## de forma mais simples
serie_diff_pd = serie_vendas - serie_vendas.shift()

In [ ]:
m_m = serie_diff_pd.rolling(window=12).mean()
plt.plot(serie_diff_pd, label='Série com Diff')
mean = plt.plot(m_m, color='red',label='Média')
plt.legend(loc='best')
plt.show()

In [ ]:
serie_diff_pd.dropna(inplace=True)
smt.graphics.plot_pacf(serie_diff_pd, lags=12, alpha=0.5)

In [ ]:
desvio = 5
random_normal = [i for i in np.random.normal(scale=desvio, size = len(serie_diff_pd))]

In [ ]:
plt.plot(serie_diff_pd.values, label='Serie Diff')
plt.plot(random_normal, color='red',label='Random estimado')
plt.legend(loc='best')
plt.show()

#### Eliminando a sazonalidade através da diferenciação

In [ ]:
serie_diff_2 = []
meses = 12
for i in range(meses, len(serie_vendas)):
    value = serie_vendas[i] - serie_vendas[i - meses]
    serie_diff_2.append(value)
serie_diff_2_pd = pd.Series(serie_diff_2)

In [ ]:
m_m = serie_diff_2_pd.rolling(window=12).mean()
plt.plot(serie_diff_2_pd, label='Série com Diff')
mean = plt.plot(m_m, color='red',label='Média')
plt.legend(loc='best')
plt.show()

In [ ]:
## de forma mais simples
serie_diff_2_pd = serie_vendas - serie_vendas.shift(12)

In [ ]:
m_m = serie_diff_2_pd.rolling(window=12).mean()
plt.plot(serie_diff_2_pd, label='Série com Diff')
mean = plt.plot(m_m, color='red',label='Média')
plt.legend(loc='best')
plt.show()

In [ ]:
serie_diff_2_pd.dropna(inplace=True)
smt.graphics.plot_pacf(serie_diff_2_pd, lags=12, alpha=0.5)

# Análisando o ruído

#### Previsão com média móveis para série diferenciada 

In [ ]:
def prev_media_mov(janela):
    prev = np.mean(janela)
    return prev
    

In [ ]:
X = serie_diff.values
tam_janela = 2
pontos_anteriores = [X[i] for i in range(tam_janela)]
pontos_futuros = [X[i] for i in range(tam_janela, len(X))]
previsao = []
target = []

for t in range(len(pontos_futuros)):
   
    janela = pontos_anteriores[-tam_janela:] #obtém a janela dos pontos anteriores 
    prev = prev_media_mov(janela)  
    previsao.append(prev)
    target.append(pontos_futuros[t])
    pontos_anteriores.append(pontos_futuros[t])

In [ ]:
plt.plot(previsao, color='red', label='MM')
plt.plot(target, label='Série Diff')
plt.legend(loc='best')
plt.show()

#### Gerando o ruído

In [ ]:
ruido = np.subtract(target,previsao)

In [ ]:
plt.plot(ruido, label='ruído')
plt.legend()
plt.show()

#### Análisando através de correlograma

In [ ]:
smt.graphics.plot_pacf(ruido, lags=12, alpha=0.5)

#### Análisando através de histograma 

* __utilizar para entender as distribuições dos dados__ 
* __No melhor caso, o ruído tem uma distribuição normal em torno de zero__ 

In [ ]:
plt.hist(ruido,  label='ruído')
plt.show()

#### Análisando através de Q-Q Plot
* __compara duas distribuições para verificar o quão semelhante ou diferentes elas são;__
* __frequentemente utilizado para verificar normalidade;__ 


In [ ]:
from statsmodels.graphics.gofplots import qqplot

In [ ]:

qqplot(ruido, line='s')
plt.show()

In [ ]:
# em caso de ser um ruído branco
e = np.random.normal(size=len(ruido))
qqplot(e, line='s')
plt.show()

## Análisando e prevendo uma série com random walk

In [ ]:
serie_microsoft = pd.read_csv('msft.txt', header=None) 

In [ ]:
plt.plot(serie_microsoft)
plt.show()

### Verificando se existe o comportamento de random walk

In [ ]:
smt.graphics.plot_pacf(serie_microsoft, lags=12, alpha=0.5)

In [ ]:
### verificando qual o passo do random walk
passos = []
for i in range(1, len(serie_microsoft)):
    passo = serie_microsoft.values[i] - serie_microsoft.values[i-1]
    passos.append(passo[0])

In [ ]:
media = np.mean(passos)

In [ ]:
desvio = np.std(passos)

In [ ]:
def prever_randomwalk(valores_anteriores, media, desvio):
    soma = 0
    for i in range(0, len(valores_anteriores)):
        soma = soma + valores_anteriores[i] + np.random.normal(media, desvio, 1)[0]
        
    prev = soma
    return prev
    
    

In [ ]:
previsoes = []
qtd_lag = 1
x_entrada = serie_microsoft[0:-qtd_lag]
y_saida = serie_microsoft[qtd_lag:]




In [ ]:
y_saida.head(5)

In [ ]:
previsao = []
for i in range(0, len(y_saida)):
    prev = prever_randomwalk(x_entrada.values[i], media, desvio)
    previsao.append(prev)
    

In [ ]:
plt.plot(previsao, color='red', label='Previsao')
plt.plot(y_saida.values, label='Real')
plt.legend(loc='best')
plt.show()

In [ ]:
plt.plot(previsao[-100:], color='red', label='Previsao')
plt.plot(y_saida.values[-100:], label='Real')
plt.legend(loc='best')
plt.show()

### Análisando o resíduo 

In [ ]:
target = y_saida.values
target = np.reshape(target, (len(target)))



In [ ]:
residuo = target- previsao

In [ ]:
plt.plot(residuo)
plt.show()

In [ ]:
smt.graphics.plot_pacf(residuo, lags=12, alpha=0.5)

# Resumo

### >A análise de séries temporais começa através da geração do gráfico da série

### >Importância de se obter uma série estacionária e como fazer isso através das técnicas de estimação e eliminação dos componentes de tendência e sazanalidade ou através da diferenciação

### >Importância da análise de ruídos e como fazer isso através de técnicas visuais 